# Programação Orientada a Objetos - Aula 6

# Atributos estáticos

- Atributo que pertence à classe, e não ao objeto instanciado daquela classe. 

Chamado também de:
- `atributo de classe` ou `variável de classe`

- Declarado diretamente na classe

```python
class Teste:
    atributo_estatico = 'nome'
```

- Acessível independente de instâncias da classe

Sintaxe para uso:

```python
nome_da_classe.nome_do_atributo
```

Exemplo de uso: Controle dos CPFs já cadastrados

In [6]:
from __future__ import annotations
from datetime import date
from typing import List, Dict

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc
        Titular.cpfs_utilizados.append(cpf)

    @property
    def nome_titular(self) -> str:
        return self._nome
    
    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc


class Conta:
    def __init__(self, titular: Titular, agencia: str, conta: str):
        self._titular: Titular = titular
        self.__agencia: str = agencia
        self.__conta: str = conta
        self._saldo: float = 0.0
        self._extrato: List[Dict[str, str]] = []

    @property
    def titular(self) -> Titular:
        return self._titular

    @property
    def agencia(self) -> str:
        return self.__agencia
    
    @property
    def conta(self) -> str:
        return self.__conta
    
    @property
    def saldo(self) -> float:
        return self._saldo

    def _adicionar_extrato(self, tipo: str, valor: float):
        valor_formatado = '{:.2f}'.format(valor)
        self._extrato.append({'key': tipo.upper(), 'value': valor_formatado})

    def _msg_resposta(self, sucesso: bool, nome_operacao: str) -> None:
        if sucesso:
            print(f'Operação realizada com sucesso. Operação: {nome_operacao}')
        else:
            print(f'Falha ao realizar operação. Operação: {nome_operacao}')

    # # posso ter métodos privados
    # def __saidas(self, valor: float, nome_operacao: str) -> bool:
    def saidas(self, valor: float, nome_operacao: str) -> bool:
        if self._saldo >= valor:
            self._saldo -= valor
            self._adicionar_extrato(tipo='s', valor=valor)
            self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
            return True
        else:
            self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)
            return False

    # # desde que tenha um método público que use ele
    # def saidas(self, valor: float, nome_operacao: str):
    #     self.__saidas(valor, nome_operacao)

    def deposito(self, valor: float) -> None:
        nome_operacao = 'Deposito'
        if valor > 0.0:
            self._saldo += valor
            self._adicionar_extrato(tipo='e', valor=valor)
            self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
        else:
            self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)

    def saque(self, valor: float) -> None:
        self._saidas(valor=valor, nome_operacao='Saque')

    def extrato(self):
        print(f'Agencia: {self.__agencia}')
        print(f'Conta: {self.__conta}')
        print(f'Titular: {self.titular._nome}')
        print(f'CPF do titular: {self._titular.cpf}')
        print('Saldo: R$', '{:.2f}'.format(self._saldo), sep=' ')
        for mov in self._extrato:
            print(f'\t{mov["key"]}: R$ {mov["value"]}')
    
class ContaCorrente(Conta):
    def __init__(self, titular: Titular, agencia: str, conta: str, tarifa: float = 15.90):
        super().__init__(titular, agencia, conta)
        self._tarifa = tarifa

    def pagamento(self, valor: float) -> None:
        self._saidas(valor=valor, nome_operacao='Pagamento')

    def transferencia(self, valor: float, conta_destino: Conta) -> None:
        nome_operacao = 'Transferencia'
        if self._saldo >= valor:
            self._saldo -= valor
            self._adicionar_extrato(tipo='s', valor=valor)
            conta_destino.deposito(valor)
            self._msg_resposta(sucesso=True, nome_operacao=nome_operacao)
        else:
            self._msg_resposta(sucesso=False, nome_operacao=nome_operacao)

    

class ContaPoupanca(Conta):
    def __init__(self, titular: Titular, agencia: str, conta: str):
        super().__init__(titular, agencia, conta)

In [5]:
dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular(nome='Pedro', cpf='40040040047', dt_nasc=dt_nasc)

Titular.cpfs_utilizados

dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular(nome='Pedro', cpf='40040040047', dt_nasc=dt_nasc)

Titular.cpfs_utilizados

['40040040047', '40040040047', '40040040047', '40040040047', '40040040047']

Como controlar efetivamente os CPFs?

Opção 1:
- Validar os CPFs no nosso programa principal

In [9]:
cpf = '40040040048'
if cpf in Titular.cpfs_utilizados:
    print('CPF já cadastrado')
else:
    dt_nasc = date(year=1991, month=8, day=6)
    t1 = Titular(nome='Pedro', cpf=cpf, dt_nasc=dt_nasc)

Titular.cpfs_utilizados

CPF já cadastrado


['40040040047',
 '40040040047',
 '40040040047',
 '40040040047',
 '40040040047',
 '40040040048']

Opção 2:
- Validar o CPF no construtor da classe Titular:

In [7]:
from datetime import date
from typing import List

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        if cpf in Titular.cpfs_utilizados:
            print('CPF já cadastrado: ', cpf)
            return
        else:
            Titular.cpfs_utilizados.append(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc
        
    @property
    def nome_titular(self) -> str:
        return self._nome
    
    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

In [5]:
cpf1 = '40040040047'
dt_nasc = date(year=1991, month=8, day=6)

t1 = Titular(nome='Pedro', cpf=cpf1, dt_nasc=dt_nasc)

cpf2 = '40040040048'
dt_nasc = date(year=1991, month=8, day=6)
t2 = Titular(nome='Pedro', cpf=cpf2, dt_nasc=dt_nasc)

t1.nome_titular
t2.nome_titular



Titular.cpfs_utilizados


# Remover return do construtor e ver se o titular é criado igual.

CPF já cadastrado:  40040040047
CPF já cadastrado:  40040040048


AttributeError: 'Titular' object has no attribute '_nome'

Problema!!!

Uso do nome da classe. Em casos de refatoração, como fazer?

```python
nome_da_classe.nome_do_atributo
```

# Métodos de classe

- Método que pertence à classe, e recebe a classe como parâmetro: `cls`

Declarado da seguinte forma:

```python
@classmethod
def metodo_de_classe(cls, parametros):
```

> Importante: nos métodos de classe passamos a referência à classe usando normalmente o `cls`

&nbsp;

Sintaxe para uso:

```python
instancia_do_objeto.nome_do_metodo_da_classe()
```

Exemplo de uso: Controle dos CPFs já cadastrados

In [19]:
from datetime import date
from sys import exception
from typing import List

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str, cpf: str, dt_nasc: date):
        if self.cpf_ja_utilizado(cpf):
            raise ValueError
        else:
            self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc
    
    @classmethod
    def cpf_ja_utilizado(cls, cpf: str) -> bool:
        if cpf in cls.cpfs_utilizados:
            print('CPF já cadastrado: ', cpf)
            return True
        return False

    @classmethod
    def adicionar_cpf(cls, cpf):
        cls.cpfs_utilizados.append(cpf)

    @property
    def nome_titular(self) -> str:
        return self._nome
    
    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

In [21]:
cpf1 = '40040040047'
dt_nasc = date(year=1991, month=8, day=6)
try:
    t1 = Titular(nome='Pedro', cpf=cpf1, dt_nasc=dt_nasc)
    t1 = Titular(nome='Pedro', cpf=cpf1, dt_nasc=dt_nasc)
except ValueError as e:
    # tratamento
    pass



cpf2 = '40040040048'
dt_nasc = date(year=1991, month=8, day=6)
t2 = Titular(nome='Pedro', cpf=cpf2, dt_nasc=dt_nasc)

CPF já cadastrado:  40040040047


Outro caso de uso: Factory

Nesse exemplo, sobrecarga de construtores.

In [22]:
from datetime import date
from sys import exception
from typing import List

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str = None, cpf: str = None, dt_nasc: date = None):
        if self.cpf_ja_utilizado(cpf):
            raise ValueError
        else:
            self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc
    
    @classmethod
    def construtor_1(cls, nome: str):
        return cls(nome=nome, cpf=None, dt_nasc=None)
    
    @classmethod
    def construtor_2(cls, nome: str, cpf: str):
        return cls(nome=nome, cpf=cpf, dt_nasc=None)

    @classmethod
    def cpf_ja_utilizado(cls, cpf: str) -> bool:
        if cpf and cpf in cls.cpfs_utilizados:
            print('CPF já cadastrado: ', cpf)
            return True
        return False

    @classmethod
    def adicionar_cpf(cls, cpf):
        cls.cpfs_utilizados.append(cpf)

    @property
    def nome_titular(self) -> str:
        return self._nome
    
    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

In [23]:
cpf1 = '40040040047'
dt_nasc = date(year=1991, month=8, day=6)
# t1 = Titular(nome='Pedro', cpf=cpf2, dt_nasc=dt_nasc) # construtor padrão
t1 = Titular.construtor_1('Pedro')
print(t1.nome_titular)
print(t1.cpf)
print(t1.data_nascimento)



Pedro
None
None


# Métodos estáticos

- Método que não deveria afetar o estado do objeto e nem da classe.
- Poderia estar fora da classe, mas por conveniência (já que normalmente envolve "assuntos" da classe) fica dentro dela.

Declarado da seguinte forma: 

```python
@staticmethod
def metodo_estatico(parametros):
```

> Importante: nos métodos estáticos, não passamos a referência ao objeto `self` e nem a referência da classe `cls`


&nbsp;

Sintaxe para uso:

```python
nome_da_classe.nome_do_metodo(parametros)
```

Exemplo de uso: Validação de CPF

In [27]:
from datetime import date
from sys import exception
from typing import List

class Titular:
    cpfs_utilizados: List[str] = []
    def __init__(self, nome: str = None, cpf: str = None, dt_nasc: date = None):
        if self.cpf_ja_utilizado(cpf):
            return
        else:
            self.adicionar_cpf(cpf)
        self._nome: str = nome
        self._cpf: str = cpf
        self._dt_nasc: date = dt_nasc
    
    @staticmethod
    def validar_cpf(cpf: str):
        if cpf and len(cpf) == 11:
            return True
        else:
            return False
        
        # mesmas validações que a de cima
        # return True if (cpf and len(cpf) == 11) else False
        # return (cpf and len(cpf) == 11)
        

    @classmethod
    def construtor_1(cls, nome: str):
        return cls(nome=nome, cpf=None, dt_nasc=None)
    
    @classmethod
    def construtor_2(cls, nome: str, cpf: str):
        return cls(nome=nome, cpf=cpf, dt_nasc=None)

    @classmethod
    def cpf_ja_utilizado(cls, cpf: str) -> bool:
        if Titular.validar_cpf(cpf) and cpf in cls.cpfs_utilizados:
            print('CPF já cadastrado: ', cpf)
            return True
        return False

    @classmethod
    def adicionar_cpf(cls, cpf):
        cls.cpfs_utilizados.append(cpf)

    @property
    def nome_titular(self) -> str:
        return self._nome
    
    @nome_titular.setter
    def nome_titular(self, nome: str) -> None:
        self._nome = nome

    @property
    def cpf(self) -> str:
        return self._cpf

    @property
    def data_nascimento(self) -> date:
        return self._dt_nasc

In [29]:
cpf1 = '40040040047'
dt_nasc = date(year=1991, month=8, day=6)
t1 = Titular(nome='Pedro', cpf=cpf1, dt_nasc=dt_nasc) # construtor padrão
t1 = Titular(nome='Pedro', cpf=cpf1, dt_nasc=dt_nasc) # construtor padrão

# ...
# ...
# ...
# ...

cpf2 = '40040040983'
print(Titular.validar_cpf(cpf2))
cpf3 = '400400409883'
print(Titular.validar_cpf(cpf3))


CPF já cadastrado:  40040040047
True
False
